In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import os
import shutil
from glob import glob
import pysumma as ps
import xarray as xr
import pandas as pd
from pathlib import Path

In [ ]:
sites = os.listdir('../sites/')
bad_sites = []
sim_sites = [s for s in sites if s not in bad_sites]

In [ ]:
flux_restart_files = [sorted(glob(f'../sites/{s}/output/flux_nn_output_restart*'))[-1] for s in sites]
flux_ic_files = [f'../sites/{s}/params/flux_initial_conditions.nc' for s in sites]

for rs, ic in zip(flux_restart_files, flux_ic_files):
    shutil.copyfile(rs, ic)

In [ ]:
state_restart_files = [sorted(glob(f'../sites/{s}/output/state_nn_output_restart*'))[-1] for s in sites]
state_ic_files = [f'../sites/{s}/params/state_initial_conditions.nc' for s in sites]

for rs, ic in zip(state_restart_files, state_ic_files):
    shutil.copyfile(rs, ic)

In [ ]:
flux_restart_files

In [ ]:
seed = 50334
np.random.seed(seed)
np.random.shuffle(sim_sites)
len(sim_sites)

nfolds = 4
kfold_test_sites = np.array(sim_sites).reshape(nfolds, -1)
kfold_train_sites = np.vstack([
    list(set(sim_sites) - set(train_sites)) for train_sites in kfold_test_sites
])
#kfold_train_sites = np.vstack([
#    list(set(sim_sites) - set(train_sites)) for train_sites in kfold_train_sites
#])
model_src = [f'../models/train_states_set_{n}.txt' for n in range(nfolds)]
model_rel_dest = [f'../params/train_states_set_{n}.txt' for n in range(nfolds)]

executable = '../state_ml_summa/bin/ml_summa'

kfold_configs = []
for n in range(nfolds) :
    kfold_sites = kfold_test_sites[n]
    config = {site: {'file_manager': f'../sites/{site}/file_manager.txt'} for site in kfold_sites}
    
    for s, c in config.items():
        # set model in file manager
        fman = ps.FileManager(c['file_manager'], name='')
        fman.options.append(ps.file_manager.FileManagerOption('neuralNetFile', model_rel_dest[n]))
        fman['outFilePrefix'] = f'state_nn_{n}_output'
        
        # save file manager
        fman.write(path=str(fman.original_path).replace('file_manager', f'file_manager_{n}_state_NN'))
        
        # copy neural net file over to params
        nn_dest = c['file_manager'].replace('file_manager.txt', model_rel_dest[n].replace('../', ''))
        shutil.copy(model_src[n], nn_dest)
        c['file_manager'] = f'../sites/{s}/file_manager_{n}_state_NN.txt'
    kfold_configs.append(config)
    
config = kfold_configs[0]
for kfc in kfold_configs[1:]:
    config.update(kfc)

In [ ]:
def spinup(sim, name, run_option, period, niters, kwargs={}):
    sim.initialize()
    sim.apply_config(kwargs.get('config', {}))
    sim.spinup(run_option, period=period, niters=niters, run_suffix=f'pysumma_spinup_{name}', **kwargs)
    sim.process = None
    return sim

In [ ]:
site = 'US-Var'

In [ ]:
config[site]

In [ ]:
orig_sim = ps.Simulation(executable, config[site]['file_manager'])
orig_sim.run('local')
print(orig_sim.status)
pre_spinup = orig_sim.output.load().copy()
orig_sim.output.close()

In [ ]:
spin = ps.Simulation(executable, config[site]['file_manager'])
spin.spinup('local', '365D', niters=10)
ic = spin.initial_conditions.load()

In [ ]:
new_sim = ps.Simulation(executable, config[site]['file_manager'])
new_sim.initial_conditions = ic
new_sim.run('local', suffix='post_spinup_nn2w')
print(new_sim.status)
post_spinup = new_sim.output.load().copy()
new_sim.output.close()

In [ ]:
(post_spinup['scalarLatHeatTotal'] -  pre_spinup['scalarLatHeatTotal']).plot()

In [ ]:
post_spinup['scalarLatHeatTotal'].plot()
pre_spinup['scalarLatHeatTotal'].plot()

In [ ]:
post_spinup['scalarSurfaceTemp'].plot()
pre_spinup['scalarSurfaceTemp'].plot()

In [ ]:
(post_spinup['scalarSurfaceTemp'] -  pre_spinup['scalarSurfaceTemp']).plot()

In [ ]:
post_spinup['scalarLatHeatTotal'].groupby(post_spinup.time.dt.week).mean().plot()
pre_spinup['scalarLatHeatTotal'].groupby(post_spinup.time.dt.week).mean().plot()